In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('coba_full.ipynb'), '..')))

In [2]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.wcs import WCS
import pandas as pd
from scipy import interpolate
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

2024-09-08 12:11:59.602727: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-08 12:12:01.910690: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-08 12:12:02.934443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-08 12:12:04.914444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-08 12:12:05.371077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-08 12:12:06.966192: I tensorflow/core/platform/cpu_feature_gu

In [3]:
#library lokal
from  utils.utilitas import inputx_outputy
from utils.arsitektur import StarNet
from utils.kma_method import KomodoMlipirAlgorithm
from utils.pre_processing import interpolate_nan, normalize_flux_minmax

In [4]:
#dapatkan data x dan y dimana x adalah flux dan y adalah vektor teff, logg, feh dan vin sin 1

name_file_1 = 'sdss_spectra_data_bintang_apogee.csv'
name_file_2 = 'file_id.csv'
folder_path = 'spectra_images_apogee'

x,y = inputx_outputy(name_file_1, name_file_2, folder_path)
print(x)
print(y)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[ 6.298077e+03  4.065951e+00  1.883649e+01  1.567700e-01]
 [ 6.296543e+03  4.057451e+00  1.998481e+01  5.374700e-02]
 [ 6.200323e+03  4.296364e+00  8.084627e+00  1.856300e-01]
 [ 6.442612e+03  4.128496e+00  7.609568e+00  1.255900e-01]
 [ 6.145681e+03  4.253140e+00  4.501839e+00 -2.147100e-01]
 [ 6.130048e+03  4.313282e+00  8.245557e+00  5.192800e-04]
 [ 6.314984e+03  4.459070e+00  8.407560e+00 -8.556100e-03]
 [ 5.177072e+03  3.332362e+00  3.241156e+00 -3.632900e-01]
 [ 6.261898e+03  4.013795e+00  6.343077e+00 -2.265800e-01]
 [ 6.159180e+03  3.716907e+00  1.168961e+01 -3.774700e-01]
 [ 5.997397e+03  4.032726e+00  9.997007e+00 -1.617900e-01]
 [ 4.956895e+03  3.029606e+00  3.247880e+00 -2.969500e-01]
 [ 6.326995e+03  4.016617e+00  1.228570e+01 -1.507200e-01]
 [ 6.124563e+03  4.263536e+00  7.650503e+

In [5]:
#interpolasi nilai nan
for i in range(len(x)):
    x[i] = interpolate_nan(x[i])

print(x)

[[ 599.06590983  599.06590983  599.06590983 ...  332.90084275
   332.90084275  332.90084275]
 [ 605.81607303  605.81607303  605.81607303 ...  383.66171023
   383.66171023  383.66171023]
 [4167.09826233 4167.09826233 4167.09826233 ... 2746.01002724
  2746.01002724 2746.01002724]
 ...
 [1554.46314469 1554.46314469 1554.46314469 ...  767.45998165
   767.45998165  767.45998165]
 [2239.04845002 2239.04845002 2239.04845002 ... 1283.75650258
  1283.75650258 1283.75650258]
 [4110.96713826 4110.96713826 4110.96713826 ... 2195.08003352
  2195.08003352 2195.08003352]]


In [6]:
#normalisasi nilai flux sehingga range serangam dari 0 - 1
for i in range(len(x)):
    x[i] = normalize_flux_minmax(x[i])

print(x)

[[0.9413752  0.9413752  0.9413752  ... 0.09535156 0.09535156 0.09535156]
 [0.91770913 0.91770913 0.91770913 ... 0.19761517 0.19761517 0.19761517]
 [0.91732356 0.91732356 0.91732356 ... 0.24926168 0.24926168 0.24926168]
 ...
 [0.73882796 0.73882796 0.73882796 ... 0.0874228  0.0874228  0.0874228 ]
 [0.80709579 0.80709579 0.80709579 ... 0.19074297 0.19074297 0.19074297]
 [0.76682514 0.76682514 0.76682514 ... 0.         0.         0.        ]]


In [7]:
#split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
#Arsitektur tanpa optimasi KMA

#Membangun model Sequential
model = Sequential()

# Conv Layer 1
model.add(Conv1D(filters=4, kernel_size=3, activation='relu', input_shape=(8575, 1)))

# Conv Layer 2
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))

# Max Pooling
model.add(MaxPooling1D(pool_size=7))

# Flattening
model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(256, activation='relu'))

# Fully Connected Layer 2
model.add(Dense(128, activation='relu'))

# Output layer untuk empat parameter bintang
model.add(Dense(4, activation='linear'))

# Kompilasi model
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

/home/khalifardy/Dokumen/ruang_kerja/code/kuliah/TA/tugas_akhir/.env/lib64/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - loss: 9007832.0000 - val_loss: 9044855.0000
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 8937908.0000 - val_loss: 9013090.0000
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 8868677.0000 - val_loss: 8962806.0000
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 8877094.0000 - val_loss: 8887043.0000
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 8858667.0000 - val_loss: 8775702.0000
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 8744664.0000 - val_loss: 8615744.0000
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 8555610.0000 - val_loss: 8390867.0000
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 8171743.0000 - val_loss: 8082415.0000
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 7988520.5000 - val_loss: 7669322.0000
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 7460798.5000 - val_loss: 7129188.0000
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 995ms/step - loss

In [9]:
#inisisai fungsi fitness

start_net = StarNet(X_train, y_train, X_test, y_test)

In [10]:
search_space = {
    'k1':[0,1],
    'k2':[0,1],
    'a1':[0,1],
    'a2':[0,1],
    'ki1':[0,1],
    'ki2':[0,1],
    'pz':[0,1],
    'AD1':[0,1],
    'AD2':[0,1],
    'AD3':[0,1],
    'KID1':[0,1],
    'KID2':[0,1],
    'KID3':[0,1],
    'lr':[0,1],
    'epoch':[0,1],
    'batch':[0,1]
}

In [12]:
#inisiasi kmh 

n = 5
p = 0.5
d = 0.1
max_iter = 100
fitness_function = start_net.fitness_function


kma = KomodoMlipirAlgorithm(n,p,d,fitness_function,search_space,max_iter)

In [13]:
kma.optimize()

/home/khalifardy/Dokumen/ruang_kerja/code/kuliah/TA/tugas_akhir/.env/lib64/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/72


2024-09-08 12:34:38.257303: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30453248 exceeds 10% of free system memory.
2024-09-08 12:34:38.352784: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29472768 exceeds 10% of free system memory.


 1/37 ━━━━━━━━━━━━━━━━━━━━ 30:50 51s/step - loss: 8909660.0000

2024-09-08 12:34:38.980755: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30453248 exceeds 10% of free system memory.
2024-09-08 12:34:39.145108: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29472768 exceeds 10% of free system memory.


 2/37 ━━━━━━━━━━━━━━━━━━━━ 26s 766ms/step - loss: 9241183.0000

2024-09-08 12:34:39.792880: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30453248 exceeds 10% of free system memory.


37/37 ━━━━━━━━━━━━━━━━━━━━ 86s 964ms/step - loss: 8889729.0000 - val_loss: 9059789.0000
Epoch 2/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 39s 906ms/step - loss: 9054065.0000 - val_loss: 9059789.0000
Epoch 3/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 32s 834ms/step - loss: 8982449.0000 - val_loss: 9059789.0000
Epoch 4/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 33s 592ms/step - loss: 8952736.0000 - val_loss: 9059789.0000
Epoch 5/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 22s 593ms/step - loss: 8917406.0000 - val_loss: 9059789.0000
Epoch 6/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 39s 552ms/step - loss: 9136560.0000 - val_loss: 9059789.0000
Epoch 7/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 555ms/step - loss: 9066500.0000 - val_loss: 9059789.0000
Epoch 8/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 558ms/step - loss: 8871911.0000 - val_loss: 9059789.0000
Epoch 9/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 18s 477ms/step - loss: 9029099.0000 - val_loss: 9059789.0000
Epoch 10/72
37/37 ━━━━━━━━━━━━━━━━━━━━ 17s 370ms/step - loss: 8920982.0000 - val_loss: 9059789.0000
Epoch 11/72
37/37 ━━